# Schedule Analysis for Optimal Attendance
### By Steven Herrera

## Introduction
The Data Science Club at the University of Houston - Clear Lake is an up-and-coming club, and, as such, giving your target audience the chance to attend is of utmost importance for success. I requested of the university a dataset of all the classes that will occur in the Spring 2022 semester so I may discover for myself what day and time may be best to hold club meetings.

In [40]:
# import libraries
import pandas as pd
import plotly.express as px

In [41]:
# load data
schedule = pd.read_csv("spring_schedule.csv",header=None)

# minor data cleaning. I drop the first row as it contains a null value for the days the class meets
schedule = schedule.drop(0)
schedule.head()

,0,1,2,3
1,F,09:00:00,09:50:00,72
2,F,11:00:00,11:50:00,22
3,M,09:00:00,09:50:00,32
4,M,13:00:00,15:50:00,89
5,M,16:00:00,18:50:00,102


In [42]:
# Having 0, 1 ,2, and 3 is not human friendly to read so I rename the columns
schedule = schedule.rename(columns={0: "Days", 1: "Start", 2:"End",3:"Enrl"})
schedule.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43 entries, 1 to 43
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Days    43 non-null     object
 1   Start   43 non-null     object
 2   End     43 non-null     object
 3   Enrl    43 non-null     int64 
dtypes: int64(1), object(3)
memory usage: 1.7+ KB


# Feature Engineering and Data Transformation

The "Start" and "End" times for this dataset is a string type so I will transform them into the more appropriate pandas datetime data type. 
The "Days" column contains all the days the class meets, however, putting these days into their column will provide for a cleaner dataset that is easier to sift through. Furthermore, Thursday is represented as "R" for classes that meet "MTWR". This will need to be accounted for when creating the "Thursday" column.
My solution is to parse through the strings and look for the letters that correspond to their respective day of the week and assign either 0 or 1.

In [43]:
# transforming string to datetime
schedule['Start'] = pd.to_datetime(schedule['Start'],format= '%H:%M:%S' ).dt.time
schedule['End'] = pd.to_datetime(schedule['End'],format= '%H:%M:%S' ).dt.time

In [44]:
# solution for looking for M, W, or F
def mwf(days, string):
    day = []
    i = 0
    for d in days:
        i+=1
        if string in d:
            day.append(1)
        else:
            day.append(0)
    return day

# solution for looking for T, TH, or R
def tth(days, string):
    day = []
    i = 0
    for d in days:
        i+=1
        if string == "T":
            if d == "TTH":
                day.append(1)
            elif "TH" in d:
                day.append(0)
            elif string in d:
                day.append(1)
            else:
                day.append(0)
                
        elif string == "R":
            if d == "TTH":
                day.append(1)
            elif "TH" in d:
                day.append(1)
            elif string in d:
                day.append(1)
            else:
                day.append(0)
                
    return day

# combining solutions
def day(days,string):
    if string in "MWF":
        return mwf(days,string)
    elif string in "TR":
        return tth(days,string)

In [45]:
# looking for the various days and putting results in a list
day_abbreviation = ['M','T','W','R','F']
results = []
for letter in day_abbreviation:
    results.append(day(schedule['Days'], letter))

In [46]:
# adding columns to the schedule dataframe
days_list = ['Monday','Tuesday','Wednesday','Thursday','Friday']

for i in range(0,len(results)):
    schedule[days_list[i]] = results[i]

In [47]:
# results of data cleaning, data transformation, and feature engineering.
schedule

,Days,Start,End,Enrl,Monday,Tuesday,Wednesday,Thursday,Friday
1,F,09:00:00,09:50:00,72,0,0,0,0,1
2,F,11:00:00,11:50:00,22,0,0,0,0,1
3,M,09:00:00,09:50:00,32,1,0,0,0,0
4,M,13:00:00,15:50:00,89,1,0,0,0,0
5,M,16:00:00,18:50:00,102,1,0,0,0,0
6,M,19:00:00,21:50:00,88,1,0,0,0,0
7,MTWR,09:00:00,09:50:00,40,1,1,1,1,0
8,MTWR,10:00:00,10:50:00,57,1,1,1,1,0
9,MTWR,12:00:00,12:50:00,40,1,1,1,1,0
10,MTWR,13:00:00,13:50:00,18,1,1,1,1,0


# Data Visualization
Now I will format the data so I can visualize when classes meet and how many students are in those classes. I choose the dates from 2021-01-24 to 2021-01-28 merely for visualization purposes as these classes will meet throughout the semester.

In [48]:
# creating a list of dataframes where each df is for each day of the week
days_DFs = []

for i in range(0,len(days_list)):
    mtwrf_df = schedule[schedule[days_list[i]] == 1]
    days_DFs.append(mtwrf_df)

In [49]:
# creating a list of dictionaries to later turn into a new dataframe
days_str = ['2021-01-24 ', '2021-01-25 ', '2021-01-26 ', '2021-01-27 ', '2021-01-28 ']
DOtW = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri']
data = []

for i in range(0, len(days_DFs)):
    s = days_str[i]
    df = days_DFs[i]
    d = DOtW[i]
    for start, end, enrl, in zip(df['Start'], df['End'], df['Enrl']):
        class_start = s + str(start)
        class_end = s + str(end)
        data.append(dict(Students = enrl, Start = class_start, End = class_end, Day = d))

In [56]:
# transform the data into a dataframe and visualize the results
df = pd.DataFrame(data)

fig = px.timeline(df, x_start="Start", x_end="End", y="Students", color = "Day",
                  title="Number of Students in Class Throughout the Week")
fig.update_traces(width=2)  #This update generates a thinner bar (like a line) instead of  the default horizontal bar
fig.show()

# Applying domain knowledge
Clearly we can see there are the least number of students in class on Fridays with the added benefit of there being large periods of time when there are no classes at all, however, students may wish to do a variety of other activities that may seem like a better option to them over attending a student organization. Therefore, I made the executive decision to rule out Fridays as a possible day on which to hold club meetings. Similarly, as Thursday will be the start of the weekend for many other students I rule out Thursday meetings as well. Upon looking at the visualization I see the optimal time to hold a club meeting is Tuesday between the hours of  12:00pm and 1:00pm. 